In [4]:
import numpy as np
from env.world import NetworkWorld as World
from env.task import TraversalTask as Task

In [5]:
W1 = {}
W1['edges'] = {0:[14,1,2,3],
               1:[0,2,3,4],
               2:[1,3,4,0],
               3:[2,4,0,1],
               4:[3,5,1,2],
               5:[4,6,7,8],
               6:[5,7,8,9],
               7:[6,8,9,5],
               8:[7,9,5,6],
               9:[8,10,6,7],
               10:[9,11,12,13],
               11:[10,12,13,14],
               12:[11,13,14,10],
               13:[12,14,10,11],
               14:[13,0,11,12]}
W1['nodes'] = np.array(list(W1['edges'].keys()))

W2 = {}
W2['edges'] = {0:[14,3], 1:[0,4], 2:[1,0],
         3:[2,1], 4:[3,2], 5:[4,8],
         6:[5,9], 7:[6,5], 8:[7,6],
         9:[8,7], 10:[9,13], 11:[10,14],
         12:[11,10], 13:[12,11], 14:[13,12]}
W2['nodes'] = np.array(list(W2['edges'].keys()))

In [6]:
w1 = World(W1['nodes'], W1['edges'])
w2 = World(W2['nodes'], W2['edges'])